# Giris
Bu çalışma, eğittiğimiz modelin sonucunu kaydetmeyi ve daha sonra farklı veri setleri üzerinde yapacağımız eğitimlerde kullanabilmeyi amaçlamaktadır.

In [1]:
# Projede kullanacagimiz kutuphaneyi indirelim
!pip install pyyaml h5py

In [2]:
# Kullanacagimiz Kutuphaneleri Yukleyelim
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.17.0


## Veri Seti
Eğitimde kullanacağımız MNIST veri kümesinin indirilmesi ve hazırlık aşamalarını gerçekleştirelim

In [3]:
(train_images, train_labels),(test_images, test_labels) = tf.keras. datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
train_labels = train_labels[:1000] # Ilk 1000 veriyi aliyoruz
test_labels = test_labels[:1000] # Ilk 1000 veriyi aliyoruz

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0 # Resimlerin boytunu (28x28) den tek boyutlu hala getiriyoruz (784), 255'e bolerek (0-255) boyutundaki pikselleri (0-1) arasina cekiyoruz.
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## Model
Kullanacağımız modeli oluşturma aşaması.

In [5]:
def create_model():
  model = tf.keras.Sequential([
      keras.layers.Dense(512, activation = 'relu', input_shape = (784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10)
  ])

  model.compile(optimizer = 'adam',
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

model = create_model()

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

# Eğitim sırasında kontrol noktalarını kaydedin
Modelimizi eğitirken bir kesinti olduğunda kaldığımız yerden devam edebilir ve kaydettiğimiz model ile yeniden eğitim yapabiliriz.

**tf.keras.callbacks.ModelCheckpoint** fonksiyonu sayesinde modelimizi hem eğitim sırasında hem de eğitim sonunda kaydedebileceğiz.

In [6]:
# Kayit edecegimiz model ciktilarinin dosyasini olusturuyrum. If komutu kullanma nedenim eger bu dosyaya benzer bir dosya var ise yeni bir tane olusturma
if not os.path.exists('/content/training_1'):
  os.mkdir('/content/training_1')

checkpoint_path = '/content/training_1/Model_save.weights.h5'
checkpoint_dir =  os.path.dirname(checkpoint_path)

# Kayit islemini yapacak fonksiyon olusturulmasi
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only = True,
                                                 verbose = 1)

# Egitim
model.fit(train_images,
          train_labels,
          epochs = 10,
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback])

Epoch 1/10
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.6203 - sparse_categorical_accuracy: 0.4996
Epoch 1: saving model to /content/training_1/Model_save.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 1.5917 - sparse_categorical_accuracy: 0.5105 - val_loss: 0.7066 - val_sparse_categorical_accuracy: 0.7750
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.4597 - sparse_categorical_accuracy: 0.8815
Epoch 2: saving model to /content/training_1/Model_save.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.4589 - sparse_categorical_accuracy: 0.8816 - val_loss: 0.5222 - val_sparse_categorical_accuracy: 0.8280
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2943 - sparse_categorical_accuracy: 0.9301
Epoch 3: saving model to /content/training_1/Model_save.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.2937 - sparse_categorical_accuracy: 0.9303 - val_loss: 0.4833 - val_sparse_categorical_accuracy: 0.8330
Epoch 4/10
32/32 ━━━━━━━━

Tutorial'da '.ckpt' formatı ile kayıt edilmiş ama yeni versiyonda 'weights.h5' olmak zorunda. Bundan dolayı eğer bir kayıt işlemi yapacak olursanız, 'KAYITISMI.weights.h5' şeklinde yapın.

In [7]:
# Olusturdugumuz dosya icerisindekileri listeler
os.listdir(checkpoint_dir)

['Model_save.weights.h5']

In [8]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print('Untrained model, accuracy:  {:5.2f}%'.format(100 * acc))

32/32 - 0s - 7ms/step - loss: 2.3779 - sparse_categorical_accuracy: 0.0900
Untrained model, accuracy:   9.00%


Yukarıdaki kodda, önceden hazırladığımız modeli yeniden oluşturduk ve bu modeli eğitim yapmadan direkt değerlendirmeye tabi tuttuk. Sonuç olarak, doğruluk oranı (accuracy) değerimiz 0.0990, yani %9'luk bir doğruluk oranına sahip.

In [9]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print('Resored model, accuracy: {:5.2f}'.format(100 * acc))

32/32 - 0s - 3ms/step - loss: 0.4007 - sparse_categorical_accuracy: 0.8750


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Resored model, accuracy: 87.50


Eğittiğimiz modelin değerlendirmesinde doğruluk oranı (accuracy) değeri 0.5390, yani %53 olarak çıktı. Eğitilmemiş model ile eğitilmiş model arasındaki doğruluk farkını gördünüz. Ayrıca modelimizi, kaydettiğimiz yoldan çağırarak değerlendirme işlemini gerçekleştirdik.

# Kontrol noktası geri arama
Yeni bir eğitim yapacağız ve bu eğitimde, her adımda kontrol noktalarında kayıt işlemi gerçekleştireceğiz

In [10]:
# Dosya olusturma
if not os.path.exists('/content/training_2'):
  os.mkdir('/content/training_2')

# Kayit yolunu belirleme
checkpoint_path = '/content/training_2/cp-{epoch:04d}.weights.h5' # Kayit edilecek model ciktilerinin yolu
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32 # her adim egitilecek veri miktari

# Toplam veri setinin batch'lere bölünmesiyle elde edilen toplam adım sayısı hesaplanır. math.ceil fonksiyonu, adım sayısının üst tam sayıya yuvarlanmasını sağlar.
import math
n_batches =len(train_images) / batch_size
n_batches = math.ceil(n_batches)

# Yeni kayit fonksiyonumuz
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path, # ğırlıkların kaydedileceği dosya yolu.
    verbose = 1, # Eğitim sırasında kaydedilen checkpoint'ler hakkında bilgi verir.
    save_weights_only = True, # Yalnızca modelin ağırlıklarının kaydedileceğini belirtir (modelin tamamı değil).
    save_freq = 5 * n_batches # Ağırlıkların her 5 batch adımında kaydedileceğini belirtir. Bu, eğitim sürecinde her 5 batch'te bir checkpoint kaydeder.
)

# Model olusturulmasi
model = create_model()

# Modelin başlangıç ağırlıkları, epoch = 0 olarak kaydedilir.
model.save_weights(checkpoint_path.format(epoch = 0))

# Egitim
model.fit(train_images,
          train_labels,
          epochs = 50,
          batch_size = batch_size,
          callbacks = [cp_callback],
          validation_data = (test_images, test_labels),
          verbose = 0)


Epoch 5: saving model to /content/training_2/cp-0005.weights.h5

Epoch 10: saving model to /content/training_2/cp-0010.weights.h5

Epoch 15: saving model to /content/training_2/cp-0015.weights.h5

Epoch 20: saving model to /content/training_2/cp-0020.weights.h5

Epoch 25: saving model to /content/training_2/cp-0025.weights.h5

Epoch 30: saving model to /content/training_2/cp-0030.weights.h5

Epoch 35: saving model to /content/training_2/cp-0035.weights.h5

Epoch 40: saving model to /content/training_2/cp-0040.weights.h5

Epoch 45: saving model to /content/training_2/cp-0045.weights.h5

Epoch 50: saving model to /content/training_2/cp-0050.weights.h5


In [11]:
os.listdir(checkpoint_dir)

['cp-0000.weights.h5',
 'cp-0045.weights.h5',
 'cp-0005.weights.h5',
 'cp-0035.weights.h5',
 'cp-0010.weights.h5',
 'cp-0050.weights.h5',
 'cp-0025.weights.h5',
 'cp-0030.weights.h5',
 'cp-0020.weights.h5',
 'cp-0040.weights.h5',
 'cp-0015.weights.h5']

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir) # TensorFlow, verilen dizinde (bu durumda checkpoint_dir), kaydedilen en son model ağırlıklarının dosya yolunu döndürür.
latest

In [13]:
model = create_model()

# model.load_weights(latest) #  normalde olmasi gereken
model.load_weights('/content/training_2/cp-0050.weights.h5')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 6ms/step - loss: 0.4911 - sparse_categorical_accuracy: 0.8750
Restored model, accuracy: 87.50%


Normalde olması gereken, 'latest' kullanarak modeli değerlendirmekti, ancak beklenmedik bir hata aldığım için ben direk en son kaydedilen model ağırlıklarının yolunu yazıyorum.

Model değerlendirmesine gelecek olursak, modelimiz accuracy (kesinlik) değeri 0.7320 yani %73 değerine ulaşmıştır.

# Bu dosyalar icerisinde neler barindiriyor?
Model eğitim sonunda ve eğitim sırasında modelin ağırlıklarını kayıt altına aldık. Bu ağırlıkları kullanarak modelimizi test işlemi de yaptık. Peki, bu ağırlıkların içerisinde neler bulunmaktadır?

- Modelin ağırlıklarını içeren bir veya birden fazla parça.
- Hangi ağırlıkların hangi parçada saklandığını gösteren bir indeks dosyası.

# Ağırlıkları manuel olarak kaydedin

Bu bölümde modelimizin ağırlıklarını manuel olarak kayıt işlemini gerçekleştireceğiz.

In [16]:
if not os.path.exists('/content/checkpoints'):
  os.mkdir('/content/checkpoints')

# Agirlikleri kaydet
model.save_weights('/content/checkpoints/my_checkpoint.weights.h5')

In [17]:
# Yeni model olustur
model = create_model()

# En son kayit ile modeli duzenlenle
model.load_weights('/content/checkpoints/my_checkpoint.weights.h5')

# Degerlendirme
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print('Duzeltilmis model, accuracy(kesinlik): {:5.2f}%'.format(100 * acc))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 - 0s - 11ms/step - loss: 0.4911 - sparse_categorical_accuracy: 0.8750
Duzeltilmis model, accuracy(kesinlik): 87.50%


En son eğittiğim modeli 'model.save_weights()' fonksiyonunu kullanarak kaydettim ve bu kaydımı yeni oluşturduğum modele güncelledim. Daha sonra değerlendirme yaparak yine aynı accuracy (kesinlik) değerine ulaştım.

# Tüm modeli kaydet
Bu bölümde tüm modelimizi kaydedeceğiz. Kayıt sırasında 'tf.keras.Model.save' fonksiyonundan yararlanacağız. Kayıt işlemini iki farklı şekilde yapacağız: .keras ve .h5 formatında kayıtlar gerçekleştireceğiz.

Not : Keras nesneleri için daha zengin, ad tabanlı kaydetme ve yeniden yükleme için yeni yüksek düzeyli .keras biçiminin kullanılması önerilir; bu, hata ayıklaması daha kolaydır. Düşük düzeyli SavedModel biçimi ve eski H5 biçimi mevcut kod için desteklenmeye devam etmektedir.

## .keras Formati

In [18]:
model = create_model()
model.fit(train_images, train_labels, epochs = 5)

# Kayit islemi her hangi bir yol belirtmeden direk isim ile kaydini gerceklestiriyoruz.
model.save('my_model.keras') # Kayit islemi

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.6287 - sparse_categorical_accuracy: 0.5008
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4840 - sparse_categorical_accuracy: 0.8581
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2656 - sparse_categorical_accuracy: 0.9330
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2111 - sparse_categorical_accuracy: 0.9529
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1444 - sparse_categorical_accuracy: 0.9729


In [19]:
new_model = tf.keras.models.load_model('my_model.keras')

new_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,221,152 (4.66 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 814,102 (3.11 MB)

In [20]:
# Modeli degerlendirme
loss, acc = new_model.evaluate(test_images, test_labels, verbose = 2)
print('Duzeltilmis model, accracy(kesinlik): {:5.2f}%'.format(100 * acc))
print(new_model.predict(test_images).shape)

32/32 - 0s - 7ms/step - loss: 0.4077 - sparse_categorical_accuracy: 0.8690
Duzeltilmis model, accracy(kesinlik): 86.90%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
(1000, 10)


## SavedModel Format
Bu yöntemi atlıyorum çünkü model.save() aşamasında .keras veya .h5 uzantısının eklenmesi gerekiyor; aksi takdirde hata alıyorum. Siz de hatayı deneyimlemek isterseniz, tutorial üzerindeki kaynak kodları kullanarak deneyebilirsiniz. [Tutorial](https://www.tensorflow.org/tutorials/keras/save_and_load)

.HDF5 Format

In [25]:
# Yeni model olustur .
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# .h5 uzantisi ule kayit islemini gerceklestir.
model.save('my_model.h5')

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.5853 - sparse_categorical_accuracy: 0.5026
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4576 - sparse_categorical_accuracy: 0.8712
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2956 - sparse_categorical_accuracy: 0.9202
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1956 - sparse_categorical_accuracy: 0.9557
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1636 - sparse_categorical_accuracy: 0.9671


In [26]:
# kayit ettigin .h5 uzantili dosyayi modele aktar
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                     │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,052 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [27]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Duzeltilmis model, accuracy(kesinlik): {:5.2f}%'.format(100 * acc))

32/32 - 0s - 14ms/step - loss: 0.4301 - sparse_categorical_accuracy: 0.8760
Duzeltilmis model, accuracy(kesinlik): 87.60%


1.'.h5' Formatı (HDF5):
  - HDF5 (Hierarchical Data Format) bir dosya formatıdır ve modelin yapısını, ağırlıkları ve eğitim bilgilerini (optimizer, loss, vs.) saklamak için yaygın olarak kullanılır.
  - Bu format, verilerin ve modellerin hızlı ve verimli bir şekilde depolanmasını sağlar.
  - .h5 formatı, hem Keras hem de TensorFlow tarafından desteklenir ve genel olarak uyumluluğu yüksektir.

2.'.keras' Formatı:
  - .keras formatı, TensorFlow 2.x ile gelen ve TensorFlow'la entegre olan bir formattır. Bu format, daha yeni bir standarttır ve TensorFlow'un modelin yapılandırmalarını, ağırlıkları ve diğer bilgileri saklama biçimi olarak kullanılabilir.
  - .keras formatı, TensorFlow 2.x'in önerdiği yerel formatıdır.

Benzerlikler:
  - Her iki format da modelin yapısını, ağırlıkları ve optimizer'ı içerir.
  - Her ikisi de modelin eğitim bilgilerini ve parametrelerini saklar, bu sayede model tekrar kullanılabilir.
  - Hem .h5 hem de .keras formatları, TensorFlow ve Keras ile uyumludur.

Farklar:
  - .h5, daha eski ve genel bir formatken, .keras daha yeni bir TensorFlow formatıdır ve yalnızca TensorFlow ile uyumlu çalışabilir.
  - .h5 formatı, daha geniş bir platformda kullanılabilirken, .keras formatı yalnızca TensorFlow ve Keras ortamlarında en iyi şekilde çalışır.
  - .h5 formatı HDF5'e dayanırken, .keras formatı doğrudan TensorFlow'un kendi dosya yapısına dayanır.

# Sonuc

Bir TensorFlow tutorial eğitim bölümünün daha sonuna geldik. Bu kısımda Save and load models(Modelleri kaydet ve yukle) kavramlarına değindik. Bundan sonra egittigimiz modelleri kayit ederek baska yerlerde kullanabilecegiz. Umarım çalışmam sizin için faydalı olmuştur. İyi günler, iyi çalışmalar dilerim.

Aşağıdaki Linklerden beni takip edebilir ve yapacağım çalışmalardan haberdar olabilirsiniz!<br>
[Linkedin](https://www.linkedin.com/in/ihsancenkiz/)<br>
[Github](https://github.com/ihsncnkz)<br>
[Kaggle](https://www.kaggle.com/ihsncnkz)